This notebook is part of [**byron v0.1**](https://github.com/microgp/byron)  
Copyright 2023 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: Apache-2.0  

In [1]:
import byron
from byron.global_symbols import *

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_37043/884074203.py:1: RuntimeWarning: 
  Paranoia checks are always enabled in notebooks: performances can be significantly impaired
  import byron


In [2]:
register = byron.f.choice_parameter([f"x{n}" for n in range(4)])
int8 = byron.f.integer_parameter(0, 2**8)
int16 = byron.f.integer_parameter(0, 2**16)

In [3]:
operations_rr = byron.f.choice_parameter(["add", "adc", "sub", "sbc", "mul"])
operations_ri = byron.f.choice_parameter(["add", "sub"])
op_rr = byron.f.macro("{op} {r1}, {r2}, {r3}", op=operations_rr, r1=register, r2=register, r3=register)
op_ri = byron.f.macro("{op} {r1}, {r2}, #{imm:#x}", op=operations_ri, r1=register, r2=register, imm=int8)

In [4]:
conditions = byron.f.choice_parameter(
    ["eq", "ne", "cs", "hs", "cc", "lo", "mi", "pl", "vs", "vc", "hi", "ls", "ge", "lt", "gt", "le", "al"]
)
branch = byron.f.macro("b{cond} {label}", cond=conditions, label=byron.f.local_reference(backward=False, loop=False))

In [5]:
prologue = byron.f.macro(
    r"""; prologue
.section	__TEXT,__text,regular,pure_instructions
.globl	_one_max    ; function one_max
.p2align	2
_one_max:
.cfi_startproc"""
)

epilogue = byron.f.macro(
    r"""; epilogue
ret
.cfi_endproc"""
)

init = byron.f.macro(
    r"""; initialization
mov x0, #{v0:#x}
mov x1, #{v1:#x}
mov x2, #{v2:#x}
mov x3, #{v3:#x}""",
    v0=int16,
    v1=int16,
    v2=int16,
    v3=int16,
)

In [6]:
core = byron.f.bunch([op_rr, op_ri, op_rr, op_ri, branch], size=(10, 20 + 1))
program = byron.f.sequence([prologue, init, core, epilogue])

In [7]:
population = byron.classes.Population(top_frame=program, fitness_function=None)
population.add_random_individual()

In [8]:
print(population.dump_individual(0, {"$dump_node_info": 1}))

; Automagically generated by MicroGP v4!2.0a0.dev1 on 12-May-2023 at 18:55:02  ;; n0 ➜ Macro❬MacroZero❭
;; n0.n1 ➜ Frame❬FrameSequence#1❭
; prologue
.section	__TEXT,__text,regular,pure_instructions
.globl	_one_max    ; function one_max
.p2align	2
_one_max:
.cfi_startproc  ;; n0.n1.n2 ➜ Macro❬UserText#1❭
; initialization
mov x0, #0x5eec
mov x1, #0xed40
mov x2, #0xa4d4
mov x3, #0xd2a0  ;; n0.n1.n3 ➜ Macro❬User#4❭
;; n0.n1.n4 ➜ Frame❬MacroBunch#1❭
add x0, x0, #0x71  ;; n0.n1.n4.n5 ➜ Macro❬User#2❭
mul x3, x0, x3  ;; n0.n1.n4.n6 ➜ Macro❬User#1❭
sub x1, x2, x0  ;; n0.n1.n4.n7 ➜ Macro❬User#1❭
bmi n18  ;; n0.n1.n4.n8 ➜ Macro❬User#3❭
add x0, x3, x1  ;; n0.n1.n4.n9 ➜ Macro❬User#1❭
sub x3, x0, #0xc7  ;; n0.n1.n4.n10 ➜ Macro❬User#2❭
add x0, x2, #0x77  ;; n0.n1.n4.n11 ➜ Macro❬User#2❭
adc x2, x2, x3  ;; n0.n1.n4.n12 ➜ Macro❬User#1❭
sbc x1, x3, x1  ;; n0.n1.n4.n13 ➜ Macro❬User#1❭
bgt n22  ;; n0.n1.n4.n14 ➜ Macro❬User#3❭
add x3, x0, #0x78  ;; n0.n1.n4.n15 ➜ Macro❬User#2❭
sub x1, x1, #0x21  ;; n0.n1.n4

In [9]:
str(population.individuals[0])

'Individual with 2 frames and 21 macros (67 parameters total, 3 structural)'

In [10]:
population.individuals[0].as_tree()

In [11]:
population.individuals[0].as_figure()